# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686549


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:59,  2.12s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.30s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:43,  1.66s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:30,  1.20s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  6.74it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  7.64it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  6.15it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.14it/s]

not-logged-in-20a275b53abec8db9f41    0.090311
not-logged-in-2b79727b9f49d75f32d0    0.000686
Dynamatt                              0.000379
suncoup                               0.000361
asucich                               0.000552
CThomas                               0.001233
jmfranci                              0.001344
acohen12                              0.006720
tosnyder                              0.000724
not-logged-in-89733ae083faff644c04    0.002039
not-logged-in-c1baf067dee2ae342bf7    0.001911
lsautter                              0.001439
not-logged-in-5f30b2bb9b7f9d49da41    0.010059
Planetme                              0.001684
not-logged-in-9f30f411df9d28d2c541    0.000757
not-logged-in-3ae935e3afaee843f530    0.000424
dinosora                              0.000379
Lavadude                              0.017971
Charred240                            0.004485
crush202020                           0.000462
fanboy_deluxe                         0.004379
jzeiszler    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())